<a href="https://colab.research.google.com/github/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/05c_BatteryESCSelection-Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Battery and ESC selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the battery and ESC selection.


![DesignGraph](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/ESCBatteryDesignGraph.png?raw=1)

> **Questions:**
* Recall the main equations to take into account during ESC and battery selection
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure

*   Main equations to take into account during ESC and battery selection :
    - V_esc =  1.84e+00*P_esc**(3.60e-01)
    - M_esc =  M_esc_ref * P_esc / P_esc_ref
    - E_bat = E_bat_ref * M_bat / M_bat_ref * 0.8 
    - C_bat = E_bat/U_bat
    - I_bat =  P_el_mot_hov * N_pro / (0.95 * U_bat) 
    - t_hov = C_bat /(I_bat*60)





### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a battery and ESC.


In [ ]:
# Specifications
N_pro=4.0#[-] Number of propellers
M_pay=1.0#[kg] Payload mass
t_hov = 3 #[h]

# Reference parameters for scaling laws
# Ref : MK-quadro
M_bat_ref = .329 # [kg] mass
E_bat_ref = 220.*3600.*.329 # [J]

# Ref : Turnigy K_Force 70HV 
P_esc_ref = 3108. # [W] Power
M_esc_ref = .115 # [kg] Mass

# Assumption
U_bat_est=14.0#[V] Battery voltage estimation
P_el_mot_hov=10.0#[W] Electrical power consumption for one motor during hover
P_el_mot_to=30.0#[W] Electrical power consumption for one motor during takeoff
U_mot_to=12.0#[V] Motor voltage during takeoff



In [ ]:
# BATTERY AND ESC
# ---   
# Battery selection & scaling laws sized from hover
    
    
    N_s_bat = ceil(U_bat_est/3.7)  # [-] Cell number, round (up value).  from math import *
    U_bat = N_s_bat *3.7 # [V] Battery voltage

    M_bat = k_mb * M_pay # [kg] Battery mass


# Hover --> autonomy
    E_bat = E_bat_ref * M_bat / M_bat_ref * 0.8  # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
    C_bat = E_bat/U_bat  # [A.s] Capacity  of the battery 
    I_bat =  P_el_mot_hov * N_pro / (0.95 * U_bat)  # [A] Current of the battery   #notebook 2
    t_hov = C_bat /(I_bat*60)  # [min] Hover time 


 #ESC    
  
    P_esc = P_el_mot_to* (U_bat/U_mot_to) # [W] power electronic power (corner power or apparent power)
    M_esc =  M_esc_ref * P_esc / P_esc_ref  # [kg] Mass ESC
    V_esc =  1.84e+00*P_esc**(3.60e-01) # [V] ESC voltage  